In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

win_name = 'back_projection'
img = cv2.imread('../img/pump_horse.jpg')
hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
draw = img.copy()

#--⑤ 역투영된 결과를 마스킹해서 결과를 출력하는 공통함수
def masking(bp, win_name):
    disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    cv2.filter2D(bp,-1,disc,bp)
    _, mask = cv2.threshold(bp, 1, 255, cv2.THRESH_BINARY)
    result = cv2.bitwise_and(img, img, mask=mask)
    cv2.imshow(win_name, result)

#--⑥ 직접 구현한 역투영 함수
def backProject_manual(hist_roi):
    #--⑦ 전체 영상에 대한 H,S 히스토그램 계산
    hist_img = cv2.calcHist([hsv_img], [0,1], None,[180,256], [0,180,0,256])
    #--⑧ 선택영역과 전체 영상에 대한 히스토그램 그램 비율계산
    hist_rate = hist_roi/ (hist_img + 1)
    #--⑨ 비율에 맞는 픽셀 값 매핑
    h,s,v = cv2.split(hsv_img)
    bp = hist_rate[h.ravel(), s.ravel()]

    bp = np.minimum(bp, 1)
    bp = bp.reshape(hsv_img.shape[:2])
    cv2.normalize(bp,bp, 0, 255, cv2.NORM_MINMAX)
    bp = bp.astype(np.uint8)
    #--⑩ 역 투영 결과로 마스킹해서 결과 출력
    masking(bp,'result_manual')
 
# OpenCV API로 구현한 함수 ---⑪ 
def backProject_cv(hist_roi):
    # 역투영 함수 호출 ---⑫
    bp = cv2.calcBackProject([hsv_img], [0, 1], hist_roi,  [0, 180, 0, 256], 1)
    # 역 투영 결과로 마스킹해서 결과 출력 ---⑬ 
    masking(bp,'result_cv')

# ROI 선택 ---①
(x,y,w,h) = cv2.selectROI(win_name, img, False)
if w > 0 and h > 0:
    #roi = draw[y:y+h, x:x+w]
    roi = img[y:y+h, x:x+w]
    cv2.rectangle(draw, (x, y), (x+w, y+h), (0,0,255), 2)
    #--② 선택한 ROI를 HSV 컬러 스페이스로 변경
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    #--③ H,S 채널에 대한 히스토그램 계산
    hist_roi = cv2.calcHist([hsv_roi],[0, 1], None, [180, 256], [0, 180, 0, 256] )
    #--④ ROI의 히스토그램을 매뉴얼 구현함수와 OpenCV 이용하는 함수에 각각 전달
    backProject_manual(hist_roi)
    backProject_cv(hist_roi)
cv2.imshow(win_name, draw)
cv2.waitKey()
cv2.destroyAllWindows()

In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

win_name = 'back_projection'  # 윈도우 창 이름 설정
img = cv2.imread('../img/cho.jpg')  # 이미지를 읽어옴
hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)  # 이미지를 HSV 색상 공간으로 변환
draw = img.copy()  # 원본 이미지를 복사하여 draw에 저장

#--⑤ 역투영된 결과를 마스킹해서 결과를 출력하는 공통함수
def masking(bp, win_name):
    disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))  # 모폴로지 연산을 위한 타원형 구조 요소 생성
    cv2.filter2D(bp, -1, disc, bp)  # 필터링 연산으로 노이즈 제거
    _, mask = cv2.threshold(bp, 1, 255, cv2.THRESH_BINARY)  # 이진화로 마스크 생성
    result = cv2.bitwise_and(img, img, mask=mask)  # 마스크를 사용하여 원본 이미지에서 해당 영역 추출
    cv2.imshow(win_name, result)  # 결과 이미지를 화면에 표시

#--⑥ 직접 구현한 역투영 함수
def backProject_manual(hist_roi):
    #--⑦ 전체 영상에 대한 H,S 히스토그램 계산
    hist_img = cv2.calcHist([hsv_img], [0, 1], None, [180, 256], [0, 180, 0, 256])  # 전체 이미지에 대한 히스토그램 계산
    #--⑧ 선택영역과 전체 영상에 대한 히스토그램 비율 계산
    hist_rate = hist_roi / (hist_img + 1)  # 선택한 영역과 전체 이미지의 히스토그램 비율 계산
    #--⑨ 비율에 맞는 픽셀 값 매핑
    h, s, v = cv2.split(hsv_img)  # HSV 이미지에서 H, S, V 채널을 분리
    bp = hist_rate[h.ravel(), s.ravel()]  # 히스토그램 비율을 사용하여 역투영 결과 계산

    bp = np.minimum(bp, 1)  # 역투영 결과 값을 1로 제한
    bp = bp.reshape(hsv_img.shape[:2])  # 원래 이미지 크기로 재배열
    cv2.normalize(bp, bp, 0, 255, cv2.NORM_MINMAX)  # 역투영 결과를 0-255로 정규화
    bp = bp.astype(np.uint8)  # 결과를 8비트 이미지로 변환
    #--⑩ 역 투영 결과로 마스킹해서 결과 출력
    masking(bp, 'result_manual')  # 역투영 결과를 마스킹하여 출력

# OpenCV API로 구현한 함수 ---⑪ 
def backProject_cv(hist_roi):
    # 역투영 함수 호출 ---⑫
    bp = cv2.calcBackProject([hsv_img], [0, 1], hist_roi, [0, 180, 0, 256], 1)  # OpenCV 함수를 사용하여 역투영 수행
    # 역 투영 결과로 마스킹해서 결과 출력 ---⑬ 
    masking(bp, 'result_cv')  # 역투영 결과를 마스킹하여 출력

# ROI 선택 ---①
(x, y, w, h) = cv2.selectROI(win_name, img, False)  # ROI(관심 영역) 선택
if w > 0 and h > 0:  # 선택된 ROI가 유효한 경우
    #roi = draw[y:y+h, x:x+w]
    roi = img[y:y+h, x:x+w]  # 선택한 ROI를 추출
    cv2.rectangle(draw, (x, y), (x+w, y+h), (0, 0, 255), 2)  # ROI 영역에 빨간색 사각형 그리기
    #--② 선택한 ROI를 HSV 컬러 스페이스로 변경
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)  # ROI를 HSV 색상 공간으로 변환
    #--③ H,S 채널에 대한 히스토그램 계산
    hist_roi = cv2.calcHist([hsv_roi], [0, 1], None, [180, 256], [0, 180, 0, 256])  # ROI의 히스토그램 계산
    #--④ ROI의 히스토그램을 매뉴얼 구현함수와 OpenCV 이용하는 함수에 각각 전달
    backProject_manual(hist_roi)  # 수동으로 구현한 역투영 함수 호출
    backProject_cv(hist_roi)  # OpenCV API를 사용한 역투영 함수 호출
cv2.imshow(win_name, draw)  # ROI가 표시된 이미지를 화면에 표시
cv2.waitKey()  # 키 입력을 기다림
cv2.destroyAllWindows()  # 모든 창 닫기
